# Chinook Business Analysis Through SQL

This project will be analyzing the Chinook database that collects data of this record store's sales. Some of the data tracked is the employee information, customer information, invoice totals, playlist information, track names and music information like album and artist names. I will be analyzing this database using SQL to answer questions pertinent to improve business.

## Connect Notebook with File

In [30]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'


The next step is to run a query that gives me information on the tables and views that already exist in the database. 


In [31]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Let's explore the album table. 

In [32]:
%%sql
SELECT *
FROM album
LIMIT 3;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2


Let's explore the artist table. 

In [33]:
%%sql 
SELECT *
FROM artist
LIMIT 3;

Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith


Let's explore the playlist_track table. 

In [34]:
%%sql
SELECT * 
FROM playlist_track
LIMIT 3;

Done.


playlist_id,track_id
1,3402
1,3389
1,3390


## The Genre that Sells the Most Tracks

The Chinook Record Store just signed a deal with a new record label that specializes in artists in the USA. We have some new album options to launch on the store but before we do so, I need to determine which genres sell the best in the USA to recommend which of the new album options we should launch with first. 

Let me look at the genre table to see what we are working with there. 

In [35]:
%%sql
SELECT *
FROM genre 
LIMIT 3;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal


To get the number of tracks sold in the USA by genre, I will have to do joins to get that metric. 

In [36]:

%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    COUNT(uts.invoice_line_id) tracks_sold,
    CAST(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


## Analyzing Employee Sales Performance

We want to find the total dollar amount of sales assigned to each sales support agent within the company.

In [37]:
%%sql 
SELECT e1.first_name || " " || e1.last_name employee_name,
       e1.title,
       e1.hire_date,
       e2.first_name || " " || e2.last_name supervisor_name,
       SUM(i.total) total_sales
FROM employee e1
LEFT JOIN employee e2 ON e1.employee_id = e2.reports_to
LEFT JOIN customer c ON c.support_rep_id = e1.employee_id
LEFT JOIN invoice i ON c.customer_id = i.customer_id
WHERE e1.title = "Sales Support Agent"
GROUP BY employee_name
ORDER BY employee_name


Done.


employee_name,title,hire_date,supervisor_name,total_sales
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,None,1731.510000000004
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,None,1584.0000000000032
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,None,1393.9200000000028


The three sales support agents were hired within a few months from each month back in 2017. Jane Peacock was the agent with the highest total number of sales with $1,731.51. Steve Johnson was the one with the lowest at $1,393.92. 

Let's explore further. How many customers did each sales support agent have? Where do most of their customers come from? How many tracks did they sell?


In [38]:
%%sql 
SELECT e.first_name || " " || e.last_name employee_name,
       e.hire_date,
       COUNT(DISTINCT(c.customer_id)) total_customers,
       SUM(i.total) total_sales
FROM employee e
LEFT JOIN customer c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice i ON c.customer_id = i.customer_id
WHERE e.title = "Sales Support Agent"
GROUP BY employee_name
ORDER BY employee_name

Done.


employee_name,hire_date,total_customers,total_sales
Jane Peacock,2017-04-01 00:00:00,21,1731.510000000004
Margaret Park,2017-05-03 00:00:00,20,1584.0000000000032
Steve Johnson,2017-10-17 00:00:00,18,1393.9200000000028


Interestingly enough, Margaret Park had the most customers. Steve Johnson had the lowest number of customers. 

## Analyzing Sales by Country

In this section, I will analyze the sales data for customers from each different country. The specific metrics I will explore by country are: 

    - total number of customers
    - total value of sales
    - average values of sales per customer
    - average order value
    
There are several countries that only have one customer. For these countries, I will be putting them in an "Other" category. 


In [39]:
%%sql

WITH single_customer_countries AS 
    (
    SELECT 
       c.country, 
       COUNT(DISTINCT c.customer_id) unique_customers,
       SUM(i.total) sales,
       COUNT(invoice_id) orders_total
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY country
HAVING unique_customers = 1
ORDER BY unique_customers
    ),
    
    other_category AS
    (
SELECT 
       "Other" country,
       SUM(unique_customers) total_customers,
       SUM(sales) total_sales ,
       SUM(sales) / SUM(unique_customers) average_sales_per_customer,
       SUM(sales) / orders_total average_order_value
FROM single_customer_countries
    ),

    remaining_countries AS
    (
    SELECT
        c.country country,
        COUNT(DISTINCT c.customer_id) total_customers,
        SUM(i.total) total_sales,
        SUM(i.total) / COUNT(DISTINCT c.customer_id) average_sales_per_customer,
        SUM(i.total) / COUNT(invoice_id) average_order_value
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY country
    HAVING total_customers <> 1
    ),
    
    compiled_countries AS 
    (
        SELECT * FROM remaining_countries
    
        UNION
    
        SELECT * FROM other_category
    )

SELECT 
    country, 
    total_customers,
    total_sales,
    average_sales_per_customer,
    average_order_value
FROM 
    (SELECT 
         cc.*,
         CASE
             WHEN cc.country = "Other" THEN 1
             ELSE 0 
             END AS sort
         FROM compiled_countries cc
    )
ORDER BY sort ASC, total_sales DESC
 





Done.


country,total_customers,total_sales,average_sales_per_customer,average_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9399999999998,72.996,109.49399999999999


## Albums vs. Individual Tracks

The Chinook Music store is set up in a way where customers can buy either a whole album or a collection of one or more individual tracks. Currently Chinook is not allowing people to buy albums and individual tracks in the same order. Management is considering a new strategy - purchase only the most popular tracks from each album from record companies instead of purchasing every track from an album. 

This section, I will be figuring out the percentage of albums vs. individual tracks sold. 

In [40]:
%%sql

WITH invoice_first_track AS 
    ( 
        SELECT 
            il.invoice_id,
            MIN(il.track_id) first_track, 
            t.album_id
        FROM invoice_line il
        INNER JOIN track t ON il.track_id = t.track_id
        GROUP BY 1
    )
    
SELECT 
    album_purchase,
    COUNT(invoice_id) invoice_count,
    CAST(COUNT(invoice_id) AS FLOAT) / ( SELECT COUNT(*)
                                         FROM invoice) percent
FROM 
    (SELECT ifs.*,
            CASE
                WHEN 
                    (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,invoice_count,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


**Recommendation:**

Given that 18.5% of sales are entire album purchases, Chinook should not phase out their current business strategy and implement the new strategy where they buy only the most popular tracks from each album from record companies. Chinook could lose up to 20% of their revenue. 